# 0.Jupyter常用快捷键

* **Shift + Enter** : 执行当前cell并选中下个cell
* **Shift + J 或 Shift + Down** : 选择下一个cell
* **Shift + K 或 Shift + Up** : 选中上一个cell
* **Shift + M** : 合并选中cell
* **Ctrl + Enter** : 执行下一个cell

* **Enter** ： 编辑模式

* **Esc** : 命令模式
* **Esc a** : 在上方新建cell
* **Esc b** : 在下方新建cell
* **Esc m** : 切换到markdown
* **Esc y** : 切换到code
* **Esc l** : 显示行数
* **Esc d d** : 删除选中的cell
* **Esc 0** : 收起、打开output


# 1.python正则库

In [2]:
# encoding: UTF-8
# py3默认编码utf-8，无需声明
import re
pattern = re.compile(r'hello.*\!')
match = pattern.match('hello, World! how are you?')
if match:
    print(match.group())

hello, World!
